In [1]:
!pip install openai
!pip install langchain_community langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.5/373.5 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.4/396.4 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.5/290.5 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 6.1 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing

In [18]:
api_key = "[YOUR-OPEN-API-KEY]"

## Chain of Thoughts Prompting

- Prefix
- Examples
- Suffix

In [2]:
examples = ["""
Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be based on {context}
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question"""]


In [3]:
import langchain

In [4]:
prefix = "You are a helpful chatbot and answer questions based on provided context only. If the answer to the question is not there in the context, you can politely say that you do not have the answer"

In [5]:
query = "In the unitary method, you first find the value of a single unit by dividing the total value by the number of units, and then multiply that by the required number of units."

context = "Example: If 10 apples cost $20, the cost of one apple is $20 divided by 10, which equals $2. Therefore, the cost of 5 apples will be 5 multiplied by $2, equaling $10."

In [6]:
example_template = """Context:{context}User:{query}AI:{answer}
"""

example_prompt = langchain.PromptTemplate(
    input_variables = ["context", "query", "answer"],
    template = example_template
)

example_prompt

PromptTemplate(input_variables=['answer', 'context', 'query'], template='Context:{context}User:{query}AI:{answer}\n')

In [7]:
suffix = """
Context:{context}
User:{query}
AI:
"""

chat_prompt = langchain.PromptTemplate.from_examples(
    examples=examples,
    suffix=suffix,
    input_variables=["context", "query"],
    prefix=prefix
)

chat_prompt

PromptTemplate(input_variables=['context', 'query'], template='You are a helpful chatbot and answer questions based on provided context only. If the answer to the question is not there in the context, you can politely say that you do not have the answer\n\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be based on {context}\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\n\nContext:{context}\nUser:{query}\nAI:\n')

In [8]:
from langchain import OpenAI

llm = OpenAI(
    model_name = "gpt-3.5-turbo-instruct",
    openai_api_key=api_key,
    temperature=0,
    max_tokens=1000
)


<ipython-input-8-34529aac1c5a>:3: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  llm = OpenAI(


In [9]:
from langchain.chains.question_answering import load_qa_chain

chain = load_qa_chain(llm,
                    chain_type="stuff",
                    prompt=chat_prompt,
                    verbose=False
                    )

<ipython-input-9-deb4441df9ce>:3: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/v0.2/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/v0.2/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/v0.2/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/v0.2/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/v0.2/docs/how_to/#qa-with-rag
  chain = load_qa_chain(llm,


In [64]:
docs = [
    langchain.schema.Document(page_content="In the unitary method, you first find the value of a single unit by dividing the total "
                          "value by the number of units, and then multiply that by the required number of units.",
             metadata={}),

    langchain.schema.Document(page_content="Example: If 10 apples cost $20, the cost of one apple is $20 divided by 10, which equals $2. "
                          "Therefore, the cost of 5 apples will be 5 multiplied by $2, equaling $10.",
             metadata={})
]

docs

[Document(page_content='In the unitary method, you first find the value of a single unit by dividing the total value by the number of units, and then multiply that by the required number of units.'),
 Document(page_content='Example: If 10 apples cost $20, the cost of one apple is $20 divided by 10, which equals $2. Therefore, the cost of 5 apples will be 5 multiplied by $2, equaling $10.')]

In [65]:
while True:
    query = input("Enter your unitary method question:\n")
    try:
        response = chain.run(input_documents=docs, query=query)
        print(response)
    except Exception as e:
        print(e)


Enter your unitary method question:
If 12 pencils cost $24, how much will 7 pencils cost?
Thought: I should first find the value of one pencil
Action: Divide the total cost by the number of pencils
Action Input: $24 / 12
Observation: Each pencil costs $2
Thought: Now I can find the cost of 7 pencils
Action: Multiply the cost of one pencil by the number of pencils required
Action Input: $2 * 7
Observation: The cost of 7 pencils is $14
Thought: I now know the final answer
Final Answer: The cost of 7 pencils will be $14.
Enter your unitary method question:
If six pencils cost Rs.500 and 3 pens cost Rs.100, how much does 2 pens and 3 pencils cost?
Thought: I should first find the value of one pencil and one pen
Action: Divide the total cost of pencils by the number of pencils and the total cost of pens by the number of pens
Action Input: Rs.500/6 and Rs.100/3
Observation: The value of one pencil is Rs.83.33 and the value of one pen is Rs.33.33
Thought: Now I can calculate the cost of 2 pen

KeyboardInterrupt: Interrupted by user

## Auto-CoT

- Automatic Chain of Thoughts

* tedious to manually give chains of thoughts, hence automatic

2 steps:
1. Question Clustering : SentenceBERT used to vectorize the questions as Q
2. Demonstration Sampling

## Getting Auto-Cot from Amazon Science

In [3]:
!git clone https://github.com/amazon-science/auto-cot.git
%cd auto-cot

Cloning into 'auto-cot'...
remote: Enumerating objects: 83, done.
remote: Counting objects: 100% (82/82), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 83 (delta 40), reused 66 (delta 35), pack-reused 1 (from 1)
Receiving objects: 100% (83/83), 36.99 KiB | 3.36 MiB/s, done.
Resolving deltas: 100% (40/40), done.
/content/auto-cot


In [4]:
!pip install -r requirements.txt

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


## Inference of Auto-Cot

In [5]:
import sys
sys.argv=['']
del sys
from api import cot

In [6]:
question  = "There were 10 friends playing a video game online when 7 players quit. If each player left had 8 lives, how many lives did they have total?"
print("Example: Zero shot:")
cot(method="zero_shot", question=question)

Example: Zero shot:
*****************************
Test Question:
There were 10 friends playing a video game online when 7 players quit. If each player left had 8 lives, how many lives did they have total?
*****************************
Prompted Input:
Q: There were 10 friends playing a video game online when 7 players quit. If each player left had 8 lives, how many lives did they have total?
A: The answer is
*****************************


APIRemovedInV1: 

You tried to access openai.Completion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


## Meta Prompting

- prioritizes form and structure over content
- uses syntax as fuiding template
- assumption of LLM having innate knowledge about the specific task or problem being addressed

- categorization of components in prompt
    - problem statements
    - solution steps
    - conclusions


### Meta prompting VS Few-shot prompting

Meta:

    - abstracts problem-solving process itself
    - creates prompts that guides model through logical steps to solve problem

Few-shot:

    - providing LLMs with small number of example problems and their solutions to learn from

In [3]:
import openai
import langchain

In [4]:
llm = langchain.OpenAI(
    model_name = "gpt-3.5-turbo-instruct",
    openai_api_key=api_key,
    temperature=0,
    max_tokens=1000
)

<ipython-input-4-b80a49dc506c>:1: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  llm = langchain.OpenAI(


In [11]:
meta_prompt = """
"Problem": "{question}",
"Solution": ["Step 1":"Begin the response with "Let's think step by step."",
    "Step 2":"Follow with the reasoning steps, ensuring the solution process is broken down clearly and logically.",
    "Step 3":"End the solution with the final answer encapsulated in a LaTeX-formatted box, ... , for clarity and emphasis.",
],
"Final Answer":"[final answer to the problem]"
    """

In [13]:
prompt = langchain.PromptTemplate(
    input_variables=["question"],
    template=meta_prompt,
    verbose=True,
)
prompt

PromptTemplate(input_variables=['question'], template='\n"Problem": "{question}",\n"Solution": ["Step 1":"Begin the response with "Let\'s think step by step."",\n    "Step 2":"Follow with the reasoning steps, ensuring the solution process is broken down clearly and logically.",\n    "Step 3":"End the solution with the final answer encapsulated in a LaTeX-formatted box, ... , for clarity and emphasis.",\n],\n"Final Answer":"[final answer to the problem]"\n    ')

In [15]:
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import LLMChain

llm_chain = LLMChain(llm=llm, prompt=prompt)


<ipython-input-15-4635a3e85d7b>:4: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  llm_chain = LLMChain(llm=llm, prompt=prompt)


In [16]:
question = "What is the derivative of x^2 + 2x with respect to x?"

In [17]:
result = llm_chain.run(
    {
        "question":question
    }
)

print("Generated output:\n", result)

<ipython-input-17-97ee6a1c37b8>:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  result = llm_chain.run(


Generated output:
 
Let's think step by step.

Step 1: The derivative of a polynomial function is found by multiplying each term by its exponent and then subtracting 1 from the exponent. 

Step 2: Applying this rule to the given function, we get:

$\frac{d}{dx}(x^2 + 2x) = 2x + 2$

Step 3: Therefore, the derivative of $x^2 + 2x$ with respect to $x$ is $\boxed{2x + 2}$.


## Hence, derivative is solved with meta prompting with step by step procedures.